In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from tensorflow.keras.models import Model
from tensorflow.keras.layers import InputLayer,Dense,Conv2D,MaxPool2D,Flatten,BatchNormalization,Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

In [ ]:
dataset,dataset_info = tfds.load('malaria',with_info=True, as_supervised=True, shuffle_files=True,split=['train'])

# DATA PREPARATION

In [ ]:
dataset

In [ ]:
TRAIN_RATIO = 0.6
VAL_RATIO = 0.2
TEST_RATIO = 0.2
DATASET_SIZE = len(dataset)

dataset = tf.data.Dataset.range(10)

train_dataset = dataset.take(int(TRAIN_RATIO * DATASET_SIZE))
val_test_dataset = dataset.skip(int(TRAIN_RATIO * DATASET_SIZE))
val_dataset = val_test_dataset.take(int(VAL_RATIO * DATASET_SIZE))
test_dataset = val_test_dataset.skip(int(VAL_RATIO * DATASET_SIZE))

# print(list(train_dataset.as_numpy_iterator()))
# dataset = tf.data.Dataset.range(10)
# dataset = dataset.skip(7)
# list(dataset.as_numpy_iterator())

In [ ]:
TRAIN_RATIO = 0.6
VAL_RATIO = 0.2
TEST_RATIO = 0.2
DATASET_SIZE = len(dataset[0])

train_dataset = dataset[0].take(int(TRAIN_RATIO * DATASET_SIZE))
val_test_dataset = dataset[0].skip(int(TRAIN_RATIO * DATASET_SIZE))
val_dataset = val_test_dataset.take(int(VAL_RATIO * DATASET_SIZE))
test_dataset = val_test_dataset.skip(int(VAL_RATIO * DATASET_SIZE))

In [ ]:
train_dataset.take(1)

In [ ]:
dataset_info

In [ ]:
for data in dataset['train'].take(1):
  print(data)

In [ ]:
for i ,(image,label) in enumerate(train_dataset.take(16)):
  ax = plt.subplot(4,4, i + 1)
  plt.title(dataset_info.features['label'].int2str(label))
  plt.imshow(image)
  plt.axis('off')

# DATA PREPROCESSING

In [ ]:
for image,label in train_dataset.take(1):
  print(image)
  print(label)

In [ ]:
IM_SIZE = 224
def resizing_rescale(image,label):
    return tf.image.resize(image,(IM_SIZE,IM_SIZE))/255.0,label

In [ ]:
train_dataset = train_dataset.map(resizing_rescale)
val_dataset = val_dataset.map(resizing_rescale)
test_dataset = test_dataset.map(resizing_rescale)

In [ ]:
train_dataset = train_dataset.shuffle(buffer_size = 8,reshuffle_each_iteration=True).batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.shuffle(buffer_size = 8,reshuffle_each_iteration=True).batch(32).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.shuffle(buffer_size = 8,reshuffle_each_iteration=True).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
print(train_dataset)
print(test_dataset)
print(val_dataset)

# SEQUENTIAL API

In [ ]:
IM_SIZE = 224
model = tf.keras.Sequential([
        InputLayer(input_shape=(IM_SIZE,IM_SIZE,3)),

        Conv2D(filters = 6, kernel_size = 3, strides = 1,padding="valid", activation = 'relu'),
        BatchNormalization(),
        MaxPool2D(pool_size = 2,strides = 2),
        Dropout(rate = 0.3),

    
        Conv2D(filters = 16, kernel_size = 3, strides = 1,padding="valid", activation = 'relu'),
        BatchNormalization(),
        MaxPool2D(pool_size = 2,strides = 2),

        Flatten(),

        Dense(100,activation = "relu"),
        BatchNormalization(),
        Dropout(rate = 0.3),

        Dense(10,activation = "relu"),
        BatchNormalization(),
        Dense(1,activation = "sigmoid"),


])
model.summary()

In [ ]:
model.compile(optimizer = Adam(learning_rate=0.1), loss = BinaryCrossentropy(),metrics = 'accuracy')

In [ ]:
history = model.fit(train_dataset,validation_data = val_dataset, epochs = 10, verbose = 1)

In [ ]:
for k,v in history.history.items():
  print(k)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Model Loss")
plt.ylabel("Loss")
plt.xlabel("EPOCHS")
plt.legend(['train_loss','val_loss'])
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Model Accuracy")
plt.ylabel("accuracy")
plt.xlabel("EPOCHS")
plt.legend(['train_acc','val_acc'])
plt.show()

# MODEL EVALUTE

In [ ]:
model.evaluate(test_dataset)

In [ ]:
model.predict(test_dataset.take(1))[0][0]

In [ ]:
def parasite_or_not(x):
  if (x < 0.5):
    return str('P')
  else:
    return str('U')

In [ ]:
for i,(image,label) in enumerate(test_dataset.take(9)):

  ax = plt.subplot(3,3,i + 1)
  plt.imshow(image[0])
  plt.title(str(parasite_or_not(label.numpy()[0])) + ":" + str(parasite_or_not(model.predict(image)[0][0])))